In [6]:
from jupyter_dash import JupyterDash

import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64


import os


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from crudFile import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser2"
password = "zion"


db = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 32448, 'AAC', 'animals')


df = pd.DataFrame.from_records(db.read({}))

df.drop(columns=['_id'],inplace=True)



#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard by Zion Johnson'))),
    html.Center(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '100px', 'height': '100px'})),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='radio-button',
            options=[
                {'label': 'Water', 'value': 'water'},
                {'label': 'Mountain', 'value': 'mountain'},
                {'label': 'Disaster', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='Water'
        ),
        html.Div(id='dd-output-container')
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6'
            ),
            html.Div(
                id='map-id',
                className='col s12 m6'
            )
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'reset':
        data = df.to_dict('records')
    else:

        filtered_data = db.filter_data(filter_type)
        if filtered_data is not None:
            data = filtered_data.to_dict('records')
        else:
            data = []
    
    return data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    
    breed_counts = dff['breed'].value_counts()
    
    pie_chart = dcc.Graph(
        figure=px.pie(names=breed_counts.index, values=breed_counts.values, title='Breed Distribution')
    )
    
    return [pie_chart]
    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(view_data, index):
    dff = pd.DataFrame.from_dict(view_data)
    if index is None:
        row = 0
    else:
        row = index[0]
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row, 4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row, 9])
                                 ])
                             ])
               ])
    ]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:8912/
